一、Dataset 的意义与常用写法
📌 作用

Dataset 是你对数据集的抽象封装，它定义了：

- 数据集有多少条（**len**）

- 每一条数据如何被读取与预处理（**getitem**）

可以用它来封装图片、文本、CSV 表格、数据库等各种形式的数据。

常见使用场景

自定义数据集（如本地图片分类）

加载标准数据集（如 torchvision.datasets.MNIST）

在 **getitem** 里做预处理（如 resize、tokenize、标准化等）


In [ ]:
from torch.utils.data import Dataset


class MyDataset(Dataset):
    def __init__(self, data, labels):
        self.data = data  # 可以是numpy数组、列表等
        self.labels = labels

    def __len__(self):
        return len(self.data)  # 返回样本数量

    def __getitem__(self, idx):
        x = self.data[idx]
        y = self.labels[idx]
        return x, y


In [ ]:
dataset = MyDataset(data=[[1, 2], [3, 4]], labels=[0, 1])
print(len(dataset))  # 2
print(dataset[0])  # ([1,2], 0)


2
([1, 2], 0)


二、DataLoader 的意义与常用写法
📌 作用

DataLoader 是对 Dataset 的进一步封装，它提供：

- 自动按 batch 分组

- 自动 shuffle 打乱

- 多进程并行加载（num_workers）

- 按 epoch 循环迭代

它返回的是一个**可迭代对象**，每次迭代返回一个 batch。

常用参数：

| 参数名        | 作用                                                        |
| ------------- | ----------------------------------------------------------- |
| `batch_size`  | 每个 batch 里有多少样本                                     |
| `shuffle`     | 每个 epoch 是否打乱数据                                     |
| `num_workers` | 数据加载使用的子进程数（>0 可以加速）                       |
| `drop_last`   | 当数据不能整除 batch_size 时是否丢弃最后一批                |
| `collate_fn`  | 自定义 batch 合并方式（默认按照索引位置分类返回）           |
| `pin_memory`  | 将数据直接写入 GPU 从内存读取数据的位置，避免中间的转化步骤 |


In [2]:
import torch
from torch.utils.data import DataLoader

data = [(x, x**2, x**3) for x in range(5)]

data

[(0, 0, 0), (1, 1, 1), (2, 4, 8), (3, 9, 27), (4, 16, 64)]

In [10]:
from typing import Tuple


def collate_fn(data):
    print("传入数据是:", data)
    x, y, z = zip(*data)
    return (x, y), z


loader = DataLoader(
    data, batch_size=2, shuffle=True, drop_last=True, collate_fn=collate_fn
)


# 自动转化为 tensor 并按照 索引位置分类返回(x, y)
for batch in loader:
    print(batch)

传入数据是: [(4, 16, 64), (0, 0, 0)]
(((4, 0), (16, 0)), (64, 0))
传入数据是: [(1, 1, 1), (3, 9, 27)]
(((1, 3), (1, 9)), (1, 27))


In [13]:
from torch.utils.data import DataLoader

# 其中的 dataset 只要实现了 __len__ & __getitem__ 就行
loader = DataLoader(
    data,
    batch_size=2,
    shuffle=True,
    num_workers=0,  # 多线程数据加载
)

for batch in loader:
    print(batch)


[tensor([3, 0]), tensor([9, 0]), tensor([27,  0])]
[tensor([4, 2]), tensor([16,  4]), tensor([64,  8])]
[tensor([1]), tensor([1]), tensor([1])]


| 项目         | Dataset                      | DataLoader                     |
| ------------ | ---------------------------- | ------------------------------ |
| 功能         | 定义数据的访问方式           | 控制批量读取、打乱、并行加载   |
| 必须实现方法 | `__len__`, `__getitem__`     | 不需要自己定义，直接传 Dataset |
| 主要用途     | 抽象原始数据（方便管理）     | 高效迭代，自动批处理           |
| 高级用法     | 自定义取样策略、动态数据增强 | 自定义 collate_fn、异步预取等  |


In [ ]:
import pandas as pd
import numpy as np
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# --- 0) 构造一个示例 DataFrame
rng = np.random.default_rng(0)
df = pd.DataFrame(
    {
        "age": rng.integers(18, 60, size=1000),
        "income": rng.normal(10_000, 3_000, size=1000),
        "city": rng.choice(["SH", "BJ", "SZ"], size=1000),
        "is_vip": rng.choice([0, 1], size=1000, p=[0.7, 0.3]),  # 作为分类标签
    }
)

# --- 1) 列划分 & 预处理（DataFrame -> 纯数值矩阵）
num_cols = ["age", "income"]
cat_cols = ["city"]
label_col = "is_vip"

# 切分数据
train_df, val_df = train_test_split(
    df, test_size=0.2, random_state=42, stratify=df[label_col]
)

# 数值列：标准化（fit on train, transform on both）
scaler = StandardScaler()
X_train_num = scaler.fit_transform(train_df[num_cols].astype("float32"))
X_val_num = scaler.transform(val_df[num_cols].astype("float32"))

# 类别列：one-hot（用 pandas 比 OneHotEncoder 版本差异更少）
X_train_cat = pd.get_dummies(train_df[cat_cols], dummy_na=True)
X_val_cat = pd.get_dummies(val_df[cat_cols], dummy_na=True)

# 对齐 one-hot 列（防止验证集缺少某些类别导致列不齐）
X_val_cat = X_val_cat.reindex(columns=X_train_cat.columns, fill_value=0)

# 合并为最终特征矩阵（np.float32）
X_train = np.hstack([X_train_num, X_train_cat.to_numpy(dtype=np.float32)]).astype(
    "float32"
)
X_val = np.hstack([X_val_num, X_val_cat.to_numpy(dtype=np.float32)]).astype("float32")

y_train = train_df[label_col].to_numpy(dtype=np.int64)  # CrossEntropyLoss 期望 long
y_val = val_df[label_col].to_numpy(dtype=np.int64)


# --- 2) 定义 Dataset：一次性把 numpy -> tensor（避免 __getitem__ 重复转换）
class TabularDataset(Dataset):
    def __init__(self, X: np.ndarray, y: np.ndarray):
        assert X.shape[0] == y.shape[0]
        # 提前转成 tensor（性能友好）：只做一次拷贝/类型转换
        self.X = torch.from_numpy(X).to(torch.float32)  # [N, D]
        self.y = torch.from_numpy(y).to(torch.long)  # [N]

    def __len__(self):
        return self.X.shape[0]

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]


train_ds = TabularDataset(X_train, y_train)
val_ds = TabularDataset(X_val, y_val)

# --- 3) DataLoader：批量、打乱、并行预取、页锁定内存
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

train_loader = DataLoader(
    train_ds,
    batch_size=128,
    shuffle=True,
    num_workers=4,
    pin_memory=True,
    persistent_workers=True,
)
val_loader = DataLoader(
    val_ds,
    batch_size=256,
    shuffle=False,
    num_workers=4,
    pin_memory=True,
    persistent_workers=True,
)

# --- 4) 一个最小可跑的模型+训练/验证循环（演示 CPU->GPU 高效搬运）
model = nn.Sequential(nn.Linear(X_train.shape[1], 64), nn.ReLU(), nn.Linear(64, 2)).to(
    device
)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

for epoch in range(3):
    model.train()
    running_loss = 0.0
    for xb, yb in train_loader:
        xb = xb.to(device, non_blocking=True)  # 配合 pin_memory=True
        yb = yb.to(device, non_blocking=True)

        optimizer.zero_grad(set_to_none=True)
        logits = model(xb)
        loss = criterion(logits, yb)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * xb.size(0)

    model.eval()
    correct = total = 0
    with torch.inference_mode():
        for xb, yb in val_loader:
            xb = xb.to(device, non_blocking=True)
            yb = yb.to(device, non_blocking=True)
            pred = model(xb).argmax(dim=1)
            correct += (pred == yb).sum().item()
            total += yb.size(0)

    print(
        f"Epoch {epoch + 1} | train_loss={running_loss / len(train_ds):.4f} "
        f"| val_acc={correct / total:.3f}"
    )
